In [1]:
from GarbageDetector import GarbageDetector 
from ultralytics import YOLO
from config import test_config
import cv2
from BatchGarbageTester import BatchGarbageTester

## 单张图片测试

自定义检测模块

In [2]:
detector = GarbageDetector(model_path=test_config['model_path'])

In [3]:
results = detector.detect(image_input=test_config['img_path'])


image 1/1 d:\Desktop\\export_oi\images\test\0ec4b1c27046c4ca.jpg: 640x480 2 Bottles, 353.5ms
Speed: 19.6ms preprocess, 353.5ms inference, 18.9ms postprocess per image at shape (1, 3, 640, 480)


In [4]:
for detection in results['detections']:
    print(f"找到 {detection['class']}, 置信度: {detection['confidence']:.2f}")

找到 瓶子, 置信度: 0.90
找到 瓶子, 置信度: 0.89


或者直接调用模型即可

In [5]:
model = YOLO(test_config['model_path'])

In [6]:
img = cv2.imread(test_config['img_path'])
result_single = model(img, conf=0.64)


0: 640x480 2 Bottles, 280.0ms
Speed: 2.4ms preprocess, 280.0ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 480)


In [7]:
for r in result_single:
    boxes = r.boxes  # 获取检测框信息（坐标、置信度、类别）
    for box in boxes:
        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()  # 检测框坐标（左上角/右下角）
        conf = box.conf[0].cpu().numpy()  # 置信度
        cls = int(box.cls[0].cpu().numpy())  # 类别ID
        cls_name = model.names[cls]  # 类别名称
        
        # 绘制检测框（可选）
        cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        cv2.putText(img, f"{cls_name} {conf:.2f}", (int(x1), int(y1)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)


In [8]:
# cv2.imshow("YOLO Result", img)
# cv2.waitKey(0)

In [9]:
# model.val()

## 批量图片测试

In [10]:
def run_comprehensive_test():
    """运行批量测试"""
    
    # 初始化测试器
    tester = BatchGarbageTester(
        model_path=test_config['model_path'],
        test_images_dir=test_config['batch_imgs_path'],  # 包含1000张测试图片的目录
        output_dir=test_config['test_output_dir']
    )
    
    # 运行批量测试
    results = tester.run_batch_test(
        confidence_threshold=0.64,
        iou_threshold=0.5,
    )
    
    # 获取摘要
    summary = tester.get_summary()
    print("\n📋 测试摘要:")
    print(f"总测试图片: {summary['total_images']}")
    print(f"总检测数量: {summary['total_detections']}")
    print(f"平均推理时间: {summary['avg_inference_time']:.3f}秒")
    print(f"平均置信度: {summary['avg_confidence']:.3f}")
    print("分类统计:", summary['category_breakdown'])
    
    return tester

In [11]:
tester = run_comprehensive_test()

🚀 开始批量测试...


测试进度: 100%|██████████| 1000/1000 [05:44<00:00,  2.91it/s, 已检测=1000, 平均时间=0.341s]


✅ 批量测试完成!
📊 生成测试报告...
📈 生成可视化图表...
📝 生成YOLO格式标注文件...
✅ 图片文件已复制到YOLO格式目录

📋 测试摘要:
总测试图片: 1000
总检测数量: 1432
平均推理时间: 0.341秒
平均置信度: 0.810
分类统计: {'厨余垃圾': 173, '有害垃圾': 169, '可回收物': 885, '其他垃圾': 205}


## 详细测试——对比标签——实际是调用val

In [ ]:
results = tester.evaluate_with_labels(
    labels_dir="export_oi/labels/",  # 真实标签目录
    data_yaml="export_oi/dataset.yaml",     # 数据集配置文件（可选）
    batch_size=16,
    imgsz=640
)

In [15]:
model.val(
    data="export_oi/test_dataset.yaml",
    imgsz=640,
    save_json=True,
    # conf=0.645,
    # iou=0.5,
    plots=True,
    split='val',
    project='test',
    name='detection_results',
    verbose=True,
)

Ultralytics 8.3.231  Python-3.12.8 torch-2.5.1 CPU (13th Gen Intel Core i5-1340P)
val: Fast image access  (ping: 0.10.1 ms, read: 399.9235.0 MB/s, size: 284.8 KB)
val: Scanning D:\Desktop\工程实训\export_oi\labels\test.cache... 1000 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 1000/1000 993.2Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 63/63 3.6s/it 3:453.6ss
                   all       1000       2772      0.395      0.396      0.372       0.29
                Bottle        262        570       0.24      0.519      0.364      0.297
                  Book        139       1068      0.412      0.377      0.296      0.185
          Mobile phone        164        221      0.846      0.796      0.856      0.733
                Banana         28         63      0.163     0.0952     0.0999     0.0563
                 Apple         72        195        0.3      0.246      0.177      0.132
                Orange     

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6, 7, 8])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000144002EA630>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.

In [16]:
model_full = YOLO('garbage_detection/yolov8s_finetuned/weights/best.pt')

In [17]:
model_full.val(
    data="export_oi/test_dataset.yaml",
    imgsz=640,
    save_json=True,
    # conf=0.645,
    # iou=0.5,
    plots=True,
    split='test',
    project='test',
    name='detection_fullmodel_results',
    verbose=True,
)

Ultralytics 8.3.231  Python-3.12.8 torch-2.5.1 CPU (13th Gen Intel Core i5-1340P)
YOLOv8s summary (fused): 72 layers, 11,129,067 parameters, 0 gradients, 28.5 GFLOPs
val: Fast image access  (ping: 0.30.1 ms, read: 13.98.9 MB/s, size: 390.8 KB)
val: Scanning D:\Desktop\工程实训\export_oi\labels\test.cache... 1000 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 1000/1000 110.9Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 63/63 3.5s/it 3:382.5ss
                   all       1000       2772      0.488        0.5      0.449      0.341
                Bottle        262        570      0.234      0.514      0.392      0.321
                  Book        139       1068      0.412      0.431      0.323      0.204
          Mobile phone        164        221      0.847      0.825      0.902      0.764
                Banana         28         63      0.381      0.302       0.26      0.164
                 Apple         72  

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6, 7, 8])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000144002F6E70>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.